In [1]:
import pandas as pd
from glob import glob
import json, requests
import os

In [2]:
repo_root = '../../'

In [3]:
raw = pd.read_csv('https://data.cdc.gov/api/views/62d6-pm5i/rows.csv?accessType=DOWNLOAD')

In [4]:
raw_update = pd.read_csv('https://data.cdc.gov/api/views/42jj-z7fa/rows.csv?accessType=DOWNLOAD')

In [5]:
raw['date'] = pd.to_datetime(raw['date'])

In [6]:
raw_update['Date'] = pd.to_datetime(raw_update['Date'])

In [7]:
raw['COUNTYFP'] = raw['FIPS_County'].astype(str)
raw['COUNTYFP'] = raw['COUNTYFP'].str.zfill(3)
raw['STATEFP'] = raw['FIPS_State'].astype(str)
raw['GEOID'] = raw['STATEFP'] + raw['COUNTYFP']

In [8]:
raw_update['COUNTYFP'] = raw_update['FIPS_Code'].astype(str)
raw_update['GEOID'] = raw_update['COUNTYFP'].str.zfill(5)

In [9]:
raw['Face_Masks_Required_in_Public'] = raw['Face_Masks_Required_in_Public'].replace("Yes", 1)
raw['Face_Masks_Required_in_Public'] = raw['Face_Masks_Required_in_Public'].replace("No", 0)
raw['Face_Masks_Required_in_Public'] = raw['Face_Masks_Required_in_Public'].fillna(-1)

In [23]:
print(raw['Face_Masks_Required_in_Public'].value_counts())

 1.0    734112
-1.0    606314
 0.0    253443
Name: Face_Masks_Required_in_Public, dtype: int64


In [10]:
# frequency count of column A
count = raw_update['Masks_Order_Code'].value_counts()
print(count)

2    2205136
1     813119
Name: Masks_Order_Code, dtype: int64


In [11]:
raw_update['Masks_Order_Code'] = raw_update['Masks_Order_Code'].fillna(-1)

In [12]:
# frequency count of column A
count = raw_update['Masks_Order_Code'].value_counts()
print(count)

2    2205136
1     813119
Name: Masks_Order_Code, dtype: int64


In [22]:
# frequency count of column A
count2 = raw_update['Face_Masks_Required_in_Public'].value_counts()
print(count2)

No public mask mandate    981693
Public mask mandate       813119
Name: Face_Masks_Required_in_Public, dtype: int64


In [24]:
raw_update['Face_Masks_Required_in_Public'] = raw_update['Face_Masks_Required_in_Public'].replace("Public mask mandate", 1)
raw_update['Face_Masks_Required_in_Public'] = raw_update['Face_Masks_Required_in_Public'].replace("No public mask mandate", 0)
raw_update['Face_Masks_Required_in_Public'] = raw_update['Face_Masks_Required_in_Public'].fillna(-1)

In [25]:
# frequency count of column A
print(raw_update['Face_Masks_Required_in_Public'].value_counts())

-1.0    1223443
 0.0     981693
 1.0     813119
Name: Face_Masks_Required_in_Public, dtype: int64


In [13]:
wide = raw.pivot(index = 'GEOID', columns = 'date', values = "Face_Masks_Required_in_Public")

In [15]:
wide.head()

date,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,...,2021-08-06,2021-08-07,2021-08-08,2021-08-09,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-14,2021-08-15
GEOID,,,,,,,,,,,,,,,,,,,,,
10001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1003,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [16]:
wide.shape

(3233, 493)

In [26]:
wide_update = raw_update.pivot(index = 'GEOID', columns = 'Date', values = "Face_Masks_Required_in_Public")

In [27]:
wide_update.head()

Date,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,...,2022-09-20,2022-09-21,2022-09-22,2022-09-23,2022-09-24,2022-09-25,2022-09-26,2022-09-27,2022-09-28,2022-09-29
GEOID,,,,,,,,,,,,,,,,,,,,,
01001,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
01003,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
01005,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
01007,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
01009,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [28]:
wide_update.shape

(3235, 933)

In [30]:
wide_update.to_csv(os.path.join(repo_root, 'public/csv/mask_mandate_cdc_update.csv'))

In [68]:
wide.to_csv(os.path.join(repo_root, 'public/csv/mask_mandate_cdc.csv'))